In [1]:
import numpy as np
from sparse_vector import *
from vector_polynomial_isomorphism import *

We note that we are using the base field `QQ` for all computations currently

In [2]:
# Let us initialize constants and vector spaces.

N = 16
n = 2
d = 3
indeterminates = [var("".join(["x_", str(i), str(j), str(k), str(l)])) for i in range(n) for j in range(n) for k in range(n) for l in range(n)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, d)

Defining x_0000, x_0001, x_0010, x_0011, x_0100, x_0101, x_0110, x_0111, x_1000, x_1001, x_1010, x_1011, x_1100, x_1101, x_1110, x_1111


In [3]:
# construct f

M = MatrixSpace(R, n^2)

x_sub = lambda i,j,k,l: R(var("".join(["x_", str(i), str(j), str(k), str(l)])))

A_12 = M(0)
A_23 = M(0)
A_13 = M(0)
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                A_12[2*i + j, 2*k + l] = x_sub(i,j,k,l)
                A_23[2*j + k, 2*i + l] = x_sub(i,j,k,l)
                A_13[2*i + k, 2*j + l] = x_sub(i,j,k,l)

f = A_12.minors(3) + A_23.minors(3) + A_13.minors(3)
f_vec = [vp_iso.polynomial_to_vector(fi) for fi in f]

In [5]:
# define W = span{f}, W_perp = orthogonal complement of span{f}

W = V.subspace(f_vec)

f_vec_basis = W.basis() # take only the linearly independent ones
grad_f = [vp_iso.vector_to_polynomial(fi).gradient() for fi in f_vec_basis]

W_perp = W.complement()

print(W)
print(W_perp)

g = W_perp.basis()
p = W.dimension()
q = W_perp.dimension()

Vector space of degree 816 and dimension 32 over Rational Field
Basis matrix:
32 x 816 dense matrix over Rational Field
Vector space of degree 816 and dimension 784 over Rational Field
Basis matrix:
784 x 816 dense matrix over Rational Field


In [6]:
# action of basis matrix E_ab in glN onto homogeneous polynomial f[i]
def E_action(a, b, i):
    sum = -1*indeterminates[b]*grad_f[i][a]
    return R(sum)

We note `M_f` $:= \tilde{M}_f$

In [7]:
# Pre-processing memoization of entries of M_f

# E_action_f[l][i][j] = (E_ij)(f_l)
E_action_f = [[[0 for j in range(N)] for i in range(N)] for l in range(p)]
for l in range(p):
    for i in range(N):
        for j in range(N):
            E_action_f[l][i][j] = sparsify(vp_iso.polynomial_to_vector(E_action(i,j,l)))

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

In [8]:
# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(p*l) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])

In [9]:
print('rank(M_f) =', M_f.rank())
print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())

rank(M_f) = 128
dim(ker(M_f)) = 128


In [10]:
print((2^2 - 1) + (2^2 - 1) + (2^2 - 1) + (2^2 - 1) +1)

13
